# L1: Information retrieval

In this lab you will apply basic techniques from information retrieval to implement the core of a minimalistic search engine. The data for this lab consists of a collection of app descriptions scraped from the [Google Play Store](https://play.google.com/store/apps?hl=en). From this collection, your search engine should retrieve those apps whose descriptions best match a given query under the vector space model.

**Before starting with this lab, make sure that you have read the [Rules for hand-in assignments](https://www.ida.liu.se/~TDDE16/exam.en.shtml#handins) and the [Policy on cheating and plagiarism](https://www.ida.liu.se/~TDDE16/exam.en.shtml#cheating).** 

## Data set

The app descriptions come in the form of a compressed [JSON](https://en.wikipedia.org/wiki/JSON) file. Start by loading this file into a [Pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe).

In [1]:
import bz2
import pandas as pd

with bz2.open('app-descriptions.json.bz2') as source:
    df = pd.read_json(source)

In Pandas, a DataFrame is a table with indexed rows and labelled columns of potentially different types. Data in a DataFrame can be accessed in various ways, including by row and by column. To give an example, the code in the next cell shows rows 200–204:

In [2]:
df[200:205]

,name,description
200,Brick Breaker Star: Space King,Introducing the best Brick Breaker game that e...
201,Brick Classic - Brick Game,Classic Brick Game!\n\nBrick Classic is a popu...
202,Bricks Breaker - Glow Balls,Bricks Breaker - Glow Balls is a addictive and...
203,Bricks Breaker Quest,How to play\n- The ball flies to wherever you ...
204,Brothers in Arms® 3,Fight brave soldiers from around the globe on ...


As you can see, there are two labelled columns: `name` (the name of the app) and `description` (a textual description). The code in the next cell shows how to acess fields from the description column.

In [3]:
df['description'][200:205]

200    Introducing the best Brick Breaker game that e...
201    Classic Brick Game!\n\nBrick Classic is a popu...
202    Bricks Breaker - Glow Balls is a addictive and...
203    How to play\n- The ball flies to wherever you ...
204    Fight brave soldiers from around the globe on ...
Name: description, dtype: object

## Problem 1: Preprocessing

Your first task is to implement a preprocessor for your search engine. In the vector space model, *preprocessing* refers to any kind of transformation that is applied before a text is vectorized. Here you can restrict yourself to a very simple preprocessing: tokenization, stop word removal, and lemmatization.

To implement your preprocessor, you can use [spaCy](https://spacy.io). Make sure that you read the [Linguistic annotations](https://spacy.io/usage/spacy-101#annotations) section of the spaCy&nbsp;101; that section contains all the information that you need for this problem (and more).

Implement your preprocessor by completing the skeleton code in the next cell, adding additional code as you feel necessary.

In [4]:
import spacy

In [11]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
def preprocess(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if not token.is_stop and token.lemma_.isalpha()]

Your implementation should conform to the following specification:

<strong>preprocess</strong> (<em>text</em>)

> Preprocesses given text by tokenizing it, removing any stop words, replacing each remaining token with its lemma (base form), and discarding all lemmas that contain non-alphabetical characters. Returns the list of remaining lemmas (represented as strings).

**Tip:** To speed up the preprocessing, you can disable loading those spaCy components that you do not need, such as the parser, and named entity recognizer. See [here](https://spacy.io/usage/processing-pipelines#disabling) for more information about this.

Test your implementation by running the following cell:

In [12]:
preprocess('Apple is looking at buying U.K. startup for $1 billion')

['Apple', 'look', 'buy', 'startup', 'billion']

This should give the following output:


## Problem 2: Vectorizing

Your next task is to vectorize the data – and more specifically, to map each app description to a tf–idf vector. For this you can use the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) class from [scikit-learn](https://scikit-learn.org/stable/). Make sure to specify your preprocessor from the previous problem as the `tokenizer` &ndash; not the `preprocessor`! &ndash; for the vectorizer. (In scikit-learn parlance, the `preprocessor` handles string-level preprocessing.)

In [7]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=preprocess)
X = vectorizer.fit_transform(df['description'])

Test your implementation by running the following cell:

In [8]:
X.shape

(1614, 21254)

This should show the dimensions of the matrix `X` to be 1614 × 21427.

**A:** We are not sure why X.shape is not returning the same dimensions. We are using spacy version > 3.0 and the preprocessing function seems to be fine too. 

## Problem 3: Retrieving

To complete the search engine, your last task is to write a function that returns the most relevant app descriptions for a given query. An easy way to do solve this task is to use scikit-learn&rsquo;s [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) class. That class implements unsupervised nearest neighbours learning, and allows you to easily find a predefined number of app descriptions whose vector representations are closest to the query vector.

In [13]:
from sklearn.neighbors import NearestNeighbors

def search(query):
    # TODO: Replace the next line with your own code.
    nn = NearestNeighbors(metric="cosine")
    nn.fit(X)
    
    kn = nn.kneighbors(vectorizer.transform([query]), 10, return_distance=False)
    return df.iloc[kn[0]]

Your implementation should conform to the following specification:

<strong>search</strong> (<em>query</em>)

> Returns the 10 app descriptions most similar (in terms of cosine similarity) to the given query as a Pandas DataFrame.

Test your implementation by running the following cell:

In [14]:
search('dodge trains')

,name,description
1301,Subway Surfers,DASH as fast as you can! \nDODGE the oncoming ...
1428,Train Conductor World,Master and manage the chaos of international r...
1394,Tiny Rails,All aboard for an adventure around the world!\...
1300,Subway Princess Runner,"Subway princess runner, Bus run, forest rush w..."
998,No Humanity - The Hardest Game,2M+ Downloads All Over The World!\n\n* IGN Nom...
1429,Train for Animals - BabyMagica free,"🚂 BabyMagica ""Train for Animals"" is a educatio..."
1168,Rush,Are you ready for a thrilling ride?\n\nRush th...
786,LEGO® DUPLO® Train,All aboard! Driving the colorful LEGO® DUPLO® ...
1465,Virus War - Space Shooting Game,Warning! Virus invasion! Destroy them with you...
1153,Road Riot,Road Riot is the global sensation that defined...


The top hit in the list should be *Subway Surfers*.

## Problem 4: Finding terms with low/high idf

Recall that the inverse document frequency (idf) of a term is the lower the more documents from a given collection the term appears in. To get a better understanding for this concept, your next task is to write code to find out which terms have the lowest/highest idf with respect to the app descriptions.

Start by sorting the terms in increasing order of idf, breaking ties by falling back on alphabetic order.

In [19]:
idx = np.argsort(vectorizer.idf_) # get inverse document frequency vector of our TfidfVectorizer object and sort by increasing order
features = vectorizer.get_feature_names() # Beware get_feature_names() is DEPRECATED and will be removed from 1.2 onwards. We will need to use get_feature_names_out([input_features]) in future updates.
terms = [features[ind] for ind in idx] 

Now, print the 10 terms with the lowest/highest idf. How do you explain the results?

In [20]:
print(terms[:10])

['game', 'play', 'feature', 'free', 'new', 'world', 'time', 'app', 'fun', 'use']


**A:** The results make sense since these are words that are expected to be found in the description of mobile games. I.e. it is very likely to find the word 'game' in a lot of documents from a given collection. 

## Problem 5: Keyword extraction

A simple method for extracting salient keywords from a document is to pick the $k$ terms with the highest tf–idf value. Your last task in this lab is to implement this method. More specifically, we ask you to implement a function `keywords` that extracts keywords from a given text.

In [37]:
def keywords(text, n=10):
    ranks = zip(vectorizer.get_feature_names(),
                 np.asarray(vectorizer.transform([text]).sum(axis=0)).ravel()) # np.asarray and .ravel() for formatting purposes
    sorted_ranks = sorted(ranks, key=lambda x: x[1], reverse=True)[:10]
    result = list(zip(*sorted_ranks))[0]
    return result

Your implementation should conform to the following specification:

<strong>keywords</strong> (<em>text</em>, <em>n</em> = 10)

> Returns a list with the $n$ (default value: 10) most salient keywords from the specified text, as measured by their tf–idf value relative to the collection of app descriptions.

Test your implementation by running the following cell:

In [38]:
print(keywords(df['description'][1428]))

('train', 'railroad', 'railway', 'rail', 'chaos', 'crash', 'haul', 'locomotive', 'overcast', 'railyard')


**A:** Some of the words differ because our approach uses alphabetical ordering if the tf-idf value is the same for certain words, while the output below doesn't have such ordering. I.e. 'haul' and 'timetable' have the same value but 'timetable' can be found in front of 'haul' (non-alphabetical ordering). 

This should give the following output:

## Reflection questions

The following reflection questions are questions that you could be asked in the oral exam. Try to answer each of them in the form of a short text and put it in the cell below. You will get feedback on your answers from your lab assistant.

**RQ 1.1:** Why do we remove common stop words and lemmatise the text? Can you give an example of a scenario where, in addition to common stopwords, there are also *domain-specific* or *application-specific* stop words?

**RQ 1.2:** In Problem&nbsp;2, what do the dimensions of the matrix `X` correspond to? This matrix gives rise to different types of *representations*. Explain what these representations are. What information from the data is preserved, what information is lost in these representations?

**RQ 1.3:** What does it mean that a term has a high/low idf value? Based on this, how can we use idf to automatically identify stop words? Why do you think is idf not used as a term weighting on its own, but always in connection with term frequency (tf–idf)?

*TODO: Enter your answers here*

**Congratulations on finishing L1! 👍**